In [1]:
import tensorflow as tf
try:
    # Disable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type != 'GPU'
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

# Fixes bad convolution
#gpus = tf.config.experimental.list_physical_devices('GPU')
#for gpu in gpus:
#    tf.config.experimental.set_memory_growth(gpu, True)

In [2]:
s = tf.io.read_file('day16.txt')

@tf.function
def solve(s):
    s = tf.strings.split(s, '\n\n')
    s = tf.strings.split(s, '\n')
    s = tf.strings.split(s, ',')
    rules = s[0].to_tensor()[:,0]
    your_ticket= tf.strings.to_number(s[1][1:].to_tensor())
    nearby_tickets= tf.strings.to_number(s[2][1:].to_tensor())
    rules = tf.strings.split(rules, ':').to_tensor()
    rule_names = rules[:,0]
    rule_values = rules[:,1]
    rule_values = tf.strings.split(rule_values, ' ').to_tensor()
    r1 = rule_values[:,1]
    r2 = rule_values[:,3]
    r1 = tf.strings.to_number(tf.strings.split(r1, '-')).to_tensor()
    r2 = tf.strings.to_number(tf.strings.split(r2, '-')).to_tensor()
    
    r = tf.transpose(tf.concat((r1, r2), 0))
    nearby_tickets_values = tf.reshape(nearby_tickets, (-1,))
    
    results = tf.vectorized_map(
        lambda v: (v >= r[0]) & (v <= r[1]),
        nearby_tickets_values
    )
    
    return tf.reduce_sum(nearby_tickets_values[~tf.reduce_any(results, 1)])

%timeit solve(s)

1.76 ms ± 134 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
s = tf.io.read_file('day16.txt')

@tf.function
def solve(s):
    s = tf.strings.split(s, '\n\n')
    s = tf.strings.split(s, '\n')
    s = tf.strings.split(s, ',')
    rules = s[0].to_tensor()[:,0]
    your_ticket= tf.strings.to_number(s[1][1:].to_tensor())
    nearby_tickets= tf.strings.to_number(s[2][1:].to_tensor())
    n_tickets = tf.shape(nearby_tickets)[0]
    rules = tf.strings.split(rules, ':').to_tensor()
    n_rules = tf.shape(rules)[0]
    rule_names = rules[:,0]
    rule_values = rules[:,1]
    rule_values = tf.strings.split(rule_values, ' ').to_tensor()
    r1 = rule_values[:,1]
    r2 = rule_values[:,3]
    r1 = tf.strings.to_number(tf.strings.split(r1, '-')).to_tensor()
    r2 = tf.strings.to_number(tf.strings.split(r2, '-')).to_tensor()
    
    all_rules = tf.transpose(tf.concat((r1, r2), 0))
    nearby_tickets_values = tf.reshape(nearby_tickets, (-1,))
    
    results = tf.vectorized_map(
        lambda v: (v >= all_rules[0]) & (v <= all_rules[1]),
        nearby_tickets_values
    )
    
    mask = tf.reduce_all(tf.reshape(tf.reduce_any(results, 1), tf.shape(nearby_tickets)), 1)
    nearby_tickets = nearby_tickets[mask]

    r = tf.stack((r1, r2), 1)
    tiled_tickets = tf.tile(nearby_tickets[:,:,None, None], (1, 1, n_rules, 2))
    
    results = (tiled_tickets >= r[:,:,0]) & (tiled_tickets <= r[:,:,1])
    # nth_ticket x nth_value x nth_rule_valid
    results = tf.reduce_any(results, -1)
    results = tf.reduce_all(results, 0)
    
    ta = tf.TensorArray(tf.int64, size=n_rules)

    for _ in tf.range(n_rules):
        row_mask = tf.math.count_nonzero(results, 1) == 1
        row_to_drop = tf.where(row_mask)[0][0]
        masked_results = results[row_mask][0]
        column_to_drop = tf.where(masked_results)[0][0]
        ta = ta.write(tf.cast(column_to_drop, tf.int32), row_to_drop)
        results &= tf.tile(~masked_results[None,:], (n_rules, 1))

    permutation = ta.stack()

    gathered = tf.gather(your_ticket[0], permutation)
    gathered = tf.cast(gathered, tf.int64)
    return tf.reduce_prod(gathered[:6])
    
%timeit solve(s)

3.7 ms ± 78.9 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


nth_ticket x nth_value x nth_rule